In [87]:
import json
from glob import glob
import os.path
import shutil
import numpy as np
import pandas as pd

In [192]:
# get nifti and json (assumes $SUBDIR/$FILES)
niiDir = r'C:\Users\je5rz\Documents\project\NICU\NII'
df = pd.DataFrame(columns=['sub','date','nifti','json','prefix'])
nifti = glob(os.path.join(niiDir,"*","*.nii*"))
for f in nifti:
    idx = int(df.shape[0])
    df.loc[idx,'nifti'] = f
    # prefix
    df.loc[idx, 'prefix'] = os.path.basename(f).replace('.nii.gz','').replace('.nii','')
    # sub and date (from filename)
    part = os.path.basename(f).split('_')
    df.loc[idx, 'sub'] = part[0]
    for p in part[2:-1]:
        if len(p) == 14 and p.isnumeric():
            df.loc[idx, 'date'] = p
    # json
    f = f.replace('.nii.gz','.json').replace('.nii','.json')
    if os.path.isfile(f):
        df.loc[idx,'json'] = f

# create visit names from unqiue dates
df_scan = df.loc[:,['sub','date']].drop_duplicates().copy().sort_values(by=['sub','date'])
prev_sub = None
for i, row in df_scan.iterrows():
    if row['sub'] != prev_sub:
        scan = 1
    else:
        scan = scan + 1    
    df_scan.loc[i,'scan'] = f'scan{scan}'

In [194]:
# deid
def json_filter(data):
    ret = False
    if 'Modality' in data and data['Modality'] == 'MR':
        if 'BodyPartExamined' in data:
            ret = True
    return ret

deid = pd.read_csv(r'C:\Users\je5rz\Documents\GitHub\uva-mri-deid\scripts\dicom_anon_fields.csv') # dicomheader fields (+ json sidecar)
# output
outDir = r'C:\Users\je5rz\Documents\project\NICU\NII_DEID'
if not os.path.isdir(outDir):
    os.makedirs(outDir)

# loop over files
for i, row in df.iterrows():
    filename = row['json']
    # remove json fields as defined in "deid"
    with open(row['json'], 'r') as f:
        data = json.load(f)
    for key in deid['name']:
        data.pop(key, None)
    # check if keeping this data
    if json_filter(data):
        # make new subject directory
        subDir = os.path.join(outDir,row['sub'])
        if not os.path.isdir(subDir):
            os.makedirs(subDir)
        # rename (remove date)
        if sum(df_scan['sub']==row['sub']) > 1: # include scan# if more than one per sub
            scan = df_scan.loc[df_scan['date']==row['date'],'scan'].values[0]
            prefix = row['prefix'].replace(f'_{row["date"]}_',f'_{scan}_')
        else:
             prefix = row['prefix'].replace(f'_{row["date"]}_','_')
        # write json
        jsonFile = os.path.join(subDir, prefix + '.json')
        with open(jsonFile, 'w') as f:
            f.write(json.dumps(data, indent=4))
        # copy nifti
        _, ext = os.path.splitext(df.loc[0,'nifti'])
        if ext == '.gz':
            ext = '.nii.gz'
        niftiFile = os.path.join(subDir, prefix + ext)
        shutil.copyfile(row['nifti'], niftiFile)
        print(f'{i+1}/{df.shape[0]}\t{prefix}')
    else:
        print(f'OMITTING:\t{row["prefix"]}')



0/1682	2731482_Coronal_diff_res_5
1/1682	2731482_Coronal_diff_res_6
2/1682	2731482_EPI_DTI_axial_(READ)_13
3/1682	2731482_EPI_DTI_axial_(READ)_14
4/1682	2731482_EPI_DTI_axial_(READ)_15
5/1682	2731482_EPI_DTI_axial_(READ)_16
6/1682	2731482_EPI_DTI_axial_(READ)_17
7/1682	2731482_localizer_2_i00001
8/1682	2731482_localizer_2_i00002
9/1682	2731482_localizer_2_i00003
10/1682	2731482_PD-T2_axial_7_e1
11/1682	2731482_PD-T2_axial_7_e2
12/1682	2731482_SWI_3D_axial_10_ph
13/1682	2731482_SWI_3D_axial_11
14/1682	2731482_SWI_3D_axial_12
15/1682	2731482_SWI_3D_axial_9
16/1682	2731482_T1_fl_axial_3
17/1682	2731482_T1_mprage_sag_100
18/1682	2731482_T1_mprage_sag_101
19/1682	2731482_T1_mprage_sag_8
20/1682	2731482_T2_coronal_4
21/1682	2731482_T2_coronal_REPEAT_19
22/1682	2734702_AAHead_Scout_1
23/1682	2734702_AAHead_Scout_2_i00001
24/1682	2734702_AAHead_Scout_2_i00002
25/1682	2734702_AAHead_Scout_2_i00003
26/1682	2734702_AAHead_Scout_2_i00004
27/1682	2734702_AAHead_Scout_3_i00001
28/1682	2734702_AAHead

277/1682	2841724_Loc_2_i00002
278/1682	2841724_Loc_2_i00003
279/1682	2841724_PD-T2_axial_9_e1
280/1682	2841724_PD-T2_axial_9_e2
281/1682	2841724_SWI_3D_axial_11
282/1682	2841724_SWI_3D_axial_12_ph
283/1682	2841724_SWI_3D_axial_13
284/1682	2841724_SWI_3D_axial_14
285/1682	2841724_T1_fl_axial_10
286/1682	2841724_T1_fl_axial_REPEAT_16
287/1682	2841724_T1_mprage_sag_6
288/1682	2841724_T1_mprage_sag_7
289/1682	2841724_T1_mprage_sag_8
290/1682	2841724_T2_coronal_5
291/1682	2854926_EPI_DTI_axial_11
292/1682	2854926_EPI_DTI_axial_12
293/1682	2854926_EPI_DTI_axial_13
294/1682	2854926_EPI_DTI_axial_14
295/1682	2854926_EPI_DTI_axial_15
296/1682	2854926_Loc_2_i00001
297/1682	2854926_Loc_2_i00002
298/1682	2854926_Loc_2_i00003
299/1682	2854926_PD-T2_axial_3_e1
300/1682	2854926_PD-T2_axial_3_e2
301/1682	2854926_SWI_3D_axial_5
302/1682	2854926_SWI_3D_axial_6_ph
303/1682	2854926_SWI_3D_axial_7
304/1682	2854926_SWI_3D_axial_8
305/1682	2854926_T1_fl_axial_4
306/1682	2854926_T1_mprage_sag_10
307/1682	2854

537/1682	2921911_3D_TOF_COW_axial_(ISO)_24_i00013
538/1682	2921911_cor_Diff_41
539/1682	2921911_cor_Diff_42
540/1682	2921911_EPI_DTI_axial_(READ)_34
541/1682	2921911_EPI_DTI_axial_(READ)_36
542/1682	2921911_EPI_DTI_axial_(READ)_37
543/1682	2921911_EPI_DTI_axial_(READ)_38
544/1682	2921911_EPI_DTI_axial_(READ)_39
545/1682	2921911_Loc_1_i00001
546/1682	2921911_Loc_1_i00002
547/1682	2921911_Loc_1_i00003
548/1682	2921911_Loc_2_i00001
549/1682	2921911_Loc_2_i00002
550/1682	2921911_Loc_2_i00003
551/1682	2921911_PD-T2_axial_3_e1
552/1682	2921911_PD-T2_axial_3_e2
553/1682	2921911_PD-T2_axial_43_e1
554/1682	2921911_PD-T2_axial_43_e2
555/1682	2921911_SWI_3D_axial_5
556/1682	2921911_SWI_3D_axial_6_ph
557/1682	2921911_SWI_3D_axial_7
558/1682	2921911_SWI_3D_axial_8
559/1682	2921911_T1_fl_axial_4
560/1682	2921911_T1_mprage_sag_10
561/1682	2921911_T1_mprage_sag_11
562/1682	2921911_T1_mprage_sag_12
563/1682	2921911_T2_coronal_9
564/1682	2927033_EPI_DTI_axial_3
565/1682	2927033_EPI_DTI_axial_4
566/1682	

784/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00004
785/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00005
786/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00006
787/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00007
788/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00008
789/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00009
790/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00010
791/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00011
792/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00012
793/1682	3017031_2D_TOF_coronal_transverse_obl_45_i00013
794/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00001
795/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00002
796/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00003
797/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00004
798/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00005
799/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00006
800/1682	3017031_2D_TOF_coronal_transverse_obl_46_i00007
801/1682	3017031_2D_TOF_coronal

1030/1682	3048010_2D_TOF_sag_sinus_obl_37_i00008
1031/1682	3048010_2D_TOF_sag_sinus_obl_37_i00009
1032/1682	3048010_2D_TOF_sag_sinus_obl_37_i00010
1033/1682	3048010_2D_TOF_sag_sinus_obl_37_i00011
1034/1682	3048010_2D_TOF_sag_sinus_obl_37_i00012
1035/1682	3048010_2D_TOF_sag_sinus_obl_37_i00013
1036/1682	3048010_2D_TOF_sag_sinus_obl_37_i00014
1037/1682	3048010_2D_TOF_sag_sinus_obl_37_i00015
1038/1682	3048010_2D_TOF_sag_sinus_obl_37_i00016
1039/1682	3048010_2D_TOF_sag_sinus_obl_37_i00017
1040/1682	3048010_2D_TOF_sag_sinus_obl_37_i00018
1041/1682	3048010_2D_TOF_sag_sinus_obl_37_i00019
1042/1682	3048010_3D_TOF_COW_axial_(ISO)_10
1043/1682	3048010_3D_TOF_COW_axial_(ISO)_11
1044/1682	3048010_3D_TOF_COW_axial_(ISO)_12
1045/1682	3048010_3D_TOF_COW_axial_(ISO)_14_i00001
1046/1682	3048010_3D_TOF_COW_axial_(ISO)_14_i00002
1047/1682	3048010_3D_TOF_COW_axial_(ISO)_14_i00003
1048/1682	3048010_3D_TOF_COW_axial_(ISO)_14_i00004
1049/1682	3048010_3D_TOF_COW_axial_(ISO)_14_i00005
1050/1682	3048010_3D_TOF_

1255/1682	3099194_T1_mprage_sag_16
1256/1682	3099194_T1_mprage_sag_18
1257/1682	3099194_T1_mprage_sag_19
1258/1682	3099194_T1_mprage_sag_8
1259/1682	3099194_T2_coronal_15
1260/1682	3110197_EPI_DTI_axial_2
1261/1682	3110197_EPI_DTI_axial_3
1262/1682	3110197_EPI_DTI_axial_4
1263/1682	3110197_EPI_DTI_axial_5
1264/1682	3110197_EPI_DTI_axial_6
1265/1682	3110197_Loc_1_i00001
1266/1682	3110197_Loc_1_i00002
1267/1682	3110197_Loc_1_i00003
1268/1682	3110197_PD-T2_axial_12_e1
1269/1682	3110197_PD-T2_axial_12_e2
1270/1682	3110197_SWI_3D_axial_14
1271/1682	3110197_SWI_3D_axial_15_ph
1272/1682	3110197_SWI_3D_axial_16
1273/1682	3110197_SWI_3D_axial_17
1274/1682	3110197_T1_fl_axial_13
1275/1682	3110197_T1_mprage_sag_10
1276/1682	3110197_T1_mprage_sag_11
1277/1682	3110197_T1_mprage_sag_8
1278/1682	3110197_T1_mprage_sag_9
1279/1682	3110197_T2_coronal_18
1280/1682	3125181_EPI_DTI_axial_3
1281/1682	3125181_EPI_DTI_axial_4
1282/1682	3125181_EPI_DTI_axial_5
1283/1682	3125181_EPI_DTI_axial_6
1284/1682	312518

1545/1682	3165664_EPI_DTI_axial_(READ)_16
1546/1682	3165664_EPI_DTI_axial_(READ)_17
1547/1682	3165664_EPI_DTI_axial_(READ)_18
1548/1682	3165664_EPI_DTI_axial_(READ)_19
1549/1682	3165664_EPI_DTI_axial_(READ)_20
1550/1682	3165664_FLAIR_axial_13
1551/1682	3165664_Loc_2_i00001
1552/1682	3165664_Loc_2_i00002
1553/1682	3165664_Loc_2_i00003
1554/1682	3165664_PD-T2_axial_6_e1
1555/1682	3165664_PD-T2_axial_6_e2
1556/1682	3165664_SWI_3D_axial_10
1557/1682	3165664_SWI_3D_axial_11
1558/1682	3165664_SWI_3D_axial_8
1559/1682	3165664_SWI_3D_axial_9_ph
1560/1682	3165664_T1_fl_axial_12
1561/1682	3165664_T1_MPRAGE_sagittal_100
1562/1682	3165664_T1_MPRAGE_sagittal_101
1563/1682	3165664_T1_MPRAGE_sagittal_7
1564/1682	3165664_T1_mprage_sag_3
1565/1682	3165664_T2_axial_14
1566/1682	3165664_T2_coronal_15
1567/1682	3167773_EPI_DTI_axial_12
1568/1682	3167773_EPI_DTI_axial_13
1569/1682	3167773_EPI_DTI_axial_14
1570/1682	3167773_EPI_DTI_axial_15
1571/1682	3167773_EPI_DTI_axial_16
1572/1682	3167773_Loc_1_i00001
1